In [1]:
!pip install twitter==2.0a2
import pandas as pd
import twitter
from time import time, sleep

In [2]:
from twitter import OAuth2

def bearers_to_apis(bearers,version) :
    """
    Entrée :
        bearers : une liste de bearers d'authentification twitter (au format string)
        version : la version de l'api twitter voulue (au format string)
    Sortie :
        apis_list : une liste d'api du module twitter
    """
    apis_list = []
    for bear in bearers : 
        b_auth = OAuth2(bearer_token=bear)
        b_to_api = twitter.Twitter(auth=b_auth,api_version=version)
        apis_list.append()
    return(apis_list)

In [58]:
def following_scrapper(ids_list,following_number,apis_list) :
    """
    Entrée :
        ids_list : la liste des ids des utilisateurs twitter dont il faut récupérer la liste des comptes following (ids au format string)
        following_number : la liste du nombre de following d'un utilisateur selon sont indice dans ids_list
        apis_list : la liste des apis de la librairie twitter, de version 1.1 dont on va se servir pour récupérer la liste des following
    Sortie : 
        dict_following : un dictionnaire où chaque clé est l'id d'un utilisateur twitter et qui renvoie la liste de ses following 
    """
    n = len(apis_list) #nombre total d'apis disponibles
    i = 0 #id de l'api utilisée
    dict_following = dict()
    for k in range(len(ids_list)) :
        if following_number[k] == 0 : #il n'y a tout simplement pas de following à récupérer
            continue
        following = [] #liste des following de l'utilisateur actuel
        page = '-1' #page de la liste des following que l'on regarde, initialement '-1'
        while page != '0' : #la page "vide" dont le numéro est 0 est celle qui conclue la liste des following
            try :
                f_request = apis_list[i].friends.ids(user_id = ids_list[k])
                if i == 0 and f_request.rate_limite_remaining == 14 : #on sauvegarde la date de la première utilisation de la première api utilisée
                    t1 = time()
                if f_request.rate_limit_remaining == 0 :
                    i = i%n + 1
                    if i == 0 : #si au moment de changer d'api, on revient à l'api initiale, on attend le temps qu'il faut pour réinitialiser le nombre de requêtes disponibles
                        t2 = time()
                        if (t2-t1) < 15*60 : #la première api n'a pas eu le temps de se réinitialiser, il va falloir attendre un peu
                            sleep(15*60 - (t2-t1)) #les fenêtres d'utilisation des requêtes sont de 15 minutes, on retranche donc le temps qu'on a déjà passé
            except : #si il y a une quelconque erreur, dans le doute on passe à l'api suivante
                i = i%n + 1
                continue
            page = f_request['next_cursor_str']
        dict_following[ids_list[k]] = following #mise à jour du dictionnaire
    return(dict_following)



In [28]:
bearer = "AAAAAAAAAAAAAAAAAAAAAK%2FKkQEAAAAABZUhkvTD68l2nJhceNyZ4alZ7qE%3Dd955XXj6P8M77bYYQjYgftta6DTUIpzovgZ5joEoVTVbdgX52U"
auth = OAuth2(bearer_token=bearer)
api = twitter.Twitter(auth=auth,api_version="1.1")

In [55]:
request = api.friends.ids(user_id = "771606164",stringify_ids = True,count = 50,cursor = '1688908768446824893')

In [56]:
request

{'ids': ['553993031',
  '2565047206',
  '1022829308',
  '1068132614417121281',
  '3427817025',
  '80847172',
  '494247530',
  '91781760',
  '708124388',
  '770736893122273281',
  '702115496',
  '18936161',
  '4877776109',
  '592946538',
  '1449700302',
  '2155215398',
  '400069650',
  '15095537',
  '15748165',
  '108568373',
  '76957331',
  '89452962',
  '987666163',
  '387170756',
  '3401605678',
  '538031518',
  '1598355026',
  '902674682',
  '496832339',
  '468198741',
  '104105131',
  '2535679632',
  '440965317',
  '1258919365',
  '154476881',
  '543582873',
  '603018371',
  '854342210',
  '728364631',
  '1561123663',
  '2333988375',
  '250650563',
  '196432825',
  '51172935',
  '98414217',
  '299780743',
  '85121813',
  '229178711',
  '1097852275'],
 'next_cursor': 0,
 'next_cursor_str': '0',
 'previous_cursor': -1674591143436015136,
 'previous_cursor_str': '-1674591143436015136',
 'total_count': None}